In [1]:
from __future__ import print_function
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
import dlib
import imgaug as ia
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils

%matplotlib inline

<h1>PyTorch</h1>

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

cuda:0


<b>Load the Model</b>

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        
        self.pool = nn.MaxPool2d(2, 2) #was Max
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=0)
        
        self.conv3 = nn.Conv2d(64, 128, kernel_size=2, stride=1, padding=1)
        
        self.conv4 = nn.Conv2d(128, 184, kernel_size=2, stride=1, padding=0)
        
        self.fc1 = nn.Linear(184 * 24 * 24, 1000)
        
        self.fc2 = nn.Linear(1000, 50)
        
        self.fc3 = nn.Linear(50, 2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = F.relu(self.conv4(x))
        x = x.view(-1, 184 * 24 * 24)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [4]:
#PATH = './fear5.pth'
#torch.save(net.state_dict(), PATH)

net = Net()

#net = torch.load("fear5.pth")
net.load_state_dict(torch.load("fear5.pth"))


<All keys matched successfully>

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

cuda:0


In [6]:
net.to(device)

Net(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(128, 184, kernel_size=(2, 2), stride=(1, 1))
  (fc1): Linear(in_features=105984, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=2, bias=True)
)

<h2>Prediction on single images</h2>

In [7]:
img = "fear4.jpg"
frame = cv2.imread(img, flags=cv2.IMREAD_COLOR)

cv2.imshow('image',frame)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [8]:
# resize frame for speed.
frame = cv2.resize(frame, (400,200), interpolation=cv2.INTER_CUBIC) #resize #originally INTER_CUBIC
frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
frame_rgb = cv2.cvtColor(frame_gray, cv2.COLOR_GRAY2RGB)

In [9]:
haar_face = "haarcascade_frontalface_alt.xml"

faceCascade = cv2.CascadeClassifier(haar_face)

detector = dlib.get_frontal_face_detector() #detects faces

nfaces = 0

In [10]:
def img_processor(frame_rgb, x, y, w, h):
    im_size = 100
    horizontal_offset = 0 
    vertical_offset = 0 
    extracted_face = frame_rgb[y+vertical_offset:y+h, x+horizontal_offset:x-horizontal_offset+w]
    extracted_face_r = cv2.resize(extracted_face, (im_size, im_size)) #resize the extracted face
    extracted_face_arr = np.array(extracted_face_r)
    normalized = np.array(extracted_face_arr, np.float32) / 255.
    normalized_arr = np.expand_dims(normalized, axis=0) #only one pic, one sample, so expand dims to turn into 4D array
    face_torch = torch.from_numpy(normalized_arr) #pytorch will only accept torch tensors
    face_torch = face_torch.reshape(1, 3, im_size, im_size) #need to reshape from (1, 100,100, 3) to (1, 3, 100, 100)
    face_torch = face_torch.to(device)
    return face_torch

In [11]:
def prediction(test_image):
    with torch.no_grad():
        net.cuda()
        output = net(test_image)
        _, predicted = torch.max(output.data, 1)
        sm = torch.nn.Softmax()
        probabilities = sm(output) 
        fear_probs = probabilities[:, 0]
        neutral_probs = probabilities[:, 1]
        return fear_probs.item(), neutral_probs.item(), predicted.item()


In [12]:
faces = faceCascade.detectMultiScale(frame_rgb, scaleFactor=1.1, minNeighbors=3, minSize=(20, 20), flags=cv2.CASCADE_SCALE_IMAGE)
nfaces=0
for (x,y,w,h) in faces:
    cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
    nfaces = nfaces + 1
    extracted_array = img_processor(frame_rgb, x, y, w, h) 
    extracted_array.to(device)
    p_fear, p_neutral, predicted_class = prediction(extracted_array.to(device)) #apply img_processor func: extracts face and resizes, converts to torch
    print(p_fear, p_neutral, predicted_class)

8.748320396989584e-07 0.9999991655349731 1
0.00120182940736413 0.998798131942749 1


/home/catherine/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  import sys


In [ ]:
#scratch

In [ ]:
item = "fear4.jpg"
im_size = 400


image = cv2.imread(item, flags=cv2.IMREAD_COLOR) #open image
face = ia.imresize_single_image(image, (im_size, im_size)) #resize
gray = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
rgb = cv2.cvtColor(gray, cv2.COLOR_GRAY2RGB)
detected_faces = faceCascade.detectMultiScale(rgb, scaleFactor=1.1, minNeighbors=3, minSize=(50, 50), flags=cv2.CASCADE_SCALE_IMAGE) #detect faces


for face in detected_faces:
    (X, Y, w, h) = face
    if w > 20:
        horizontal_offset = 0
        vertical_offset = 0
        extracted_face = rgb[Y+vertical_offset:Y+h, X+horizontal_offset:X-horizontal_offset+w]
        extracted_face_r = ia.imresize_single_image(extracted_face, (im_size, im_size)) #resize
        
        tester = img_processor(extracted_face_r, X, Y, w, h)
        tester.to(device)
        prediction = prediction(tester.to(device))
    
        print(prediction)


In [ ]:
haar_face = "haarcascade_frontalface_alt.xml"

faceCascade = cv2.CascadeClassifier(haar_face)

detector = dlib.get_frontal_face_detector() #detects faces

faces = faceCascade.detectMultiScale(frame_rgb, 1.1, scaleFactor=1.1, minNeighbors=3, minSize=(20, 20), flags=cv2.CASCADE_SCALE_IMAGE)

nfaces = 0

for (x,y,w,h) in faces:
    cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
    nfaces = nfaces + 1
        
    extracted_array = img_processor(frame_rgb, x, y, w, h) #apply img_processor func: extracts face and resizes, converts to array
        
    prediction_result = prediction(extracted_array)
    
    prediction_class = prediction_result.argmax(axis=-1)
    
    print(prediction_result)

# show result
cv2.imshow("Result",frame)